In [1]:
!tree ../dataset

../dataset
├── en_espanol
│   ├── docx2txt.py
│   ├── Второй_жанр_исходная.txt
│   └── Первый_жанр_исходная.txt
├── Второй_жанр_исходная.txt
├── Первый_жанр_исходная.txt
├── Сокращение по частям речи
│   ├── 1.Первый жанр исходная выборка.txt
│   ├── 2.Первый жанр без клауз, включающих наречия.txt
│   ├── 3.Первый жанр без клауз, включающих глаголы.txt
│   ├── 4. Первый жанр без клауз, включающих глаголы и наречия.txt
│   ├── Без прилагательных второй жанр.txt
│   ├── Без прилагательных первый жанр.txt
│   └── Случайные выборки.txt
└── сокращение по частотности
    ├── 1а_ без сокращений.txt
    ├── 1б_Изъяты лексемы с частотой выше 100.txt
    ├── 1в_Изъяты лексемы с частотой выше 49.txt
    ├── 1г_Изъяты лексемы с частотой выше 29.txt
    ├── 1д_Изъяты лексемы с частотой выше 9.txt
    ├── 1е_Изъяты лексемы с частотой выше 5.txt
    ├── 2а_ без сокращений.txt
    ├── 2б_Изъяты лексемы с частотой выше 100.txt
    ├── 2в_Изъяты лексемы с частотой выше 49.txt
    ├── 2г_Изъяты лексемы с

In [2]:
ls

'1ё_Изъяты лексемы с частотой выше 3.txt'   train.py
'2ё_Изъяты лексемы с частотой выше 3.txt'   utils.py
 __pycache__/                              'Сокращение по частям речи'/
 funciones.py                              'сокращение по частотности'/
 generador.ipynb                           'сокращение по частотности.ipynb'
 memory.py                                 'сокращение по частотности.md'
 test_gpu.py


In [ ]:
#     ├── 1а_ без сокращений.txt
#     ├── 1б_Изъяты лексемы с частотой выше 100.txt
#     ├── 1в_Изъяты лексемы с частотой выше 49.txt
#     ├── 1г_Изъяты лексемы с частотой выше 29.txt
#     ├── 1д_Изъяты лексемы с частотой выше 9.txt
#     ├── 1е_Изъяты лексемы с частотой выше 5.txt
#     ├── 1ё_Изъяты лексемы с частотой выше 3.txt
#     ├── 2а_ без сокращений.txt
#     ├── 2б_Изъяты лексемы с частотой выше 100.txt
#     ├── 2в_Изъяты лексемы с частотой выше 49.txt
#     ├── 2г_Изъяты лексемы с частотой выше 29.txt
#     ├── 2д_Изъяты лексемы с частотой выше 9.txt
#     ├── 2е_Изъяты лексемы с частотой выше 5.txt
#     └── 2ё_Изъяты лексемы с частотой выше 3.txt
import funciones
from utils import train_wrapper
import warnings
import os
# Suprimir warnings específicos
warnings.filterwarnings('ignore', category=UserWarning)  # Para sklearn y otros
warnings.filterwarnings('ignore', category=FutureWarning)  # Para huggingface y transformers
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from contextlib import redirect_stderr
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp

mp.set_start_method('spawn', force=True)

def train_wrapper(args):
    dataset, config = args  # Desempaquetar los argumentos
    return funciones.train_and_evaluate_dataset(
        dataset['path1'],
        dataset['path2'],
        config,
        dataset['name']
    )

# Uso del código
def main():
    # Crear configuración
    config = funciones.TrainingConfig(
        model_name='DeepPavlov/rubert-base-cased',
        max_length=128,
        batch_size=128,
        epochs=3,
        learning_rate=2e-5,
        num_repeats=6,
        test_size=0.2,
        threshold=0.5
    )
    
    # Lista de conjuntos de datos a procesar
    
    datasets = [
        {
            'path1': '../dataset/сокращение по частотности/1б_Изъяты лексемы с частотой выше 100.txt',
            'path2': '../dataset/сокращение по частотности/2б_Изъяты лексемы с частотой выше 100.txt',
            'name': 'Изъяты лексемы с частотой выше 100'
        },
        {
            'path1': '../dataset/сокращение по частотности/1в_Изъяты лексемы с частотой выше 49.txt',
            'path2': '../dataset/сокращение по частотности/2в_Изъяты лексемы с частотой выше 49.txt',
            'name': 'Изъяты лексемы с частотой выше 49'
        },
        {
            'path1': '../dataset/сокращение по частотности/1г_Изъяты лексемы с частотой выше 29.txt',
            'path2': '../dataset/сокращение по частотности/2г_Изъяты лексемы с частотой выше 29.txt',
            'name': 'Изъяты лексемы с частотой выше 29'
        },
        {
            'path1': '../dataset/сокращение по частотности/1д_Изъяты лексемы с частотой выше 9.txt',
            'path2': '../dataset/сокращение по частотности/2д_Изъяты лексемы с частотой выше 9.txt',
            'name': 'Изъяты лексемы с частотой выше 9'
        },
        {
            'path1': '../dataset/сокращение по частотности/1е_Изъяты лексемы с частотой выше 5.txt',
            'path2': '../dataset/сокращение по частотности/2е_Изъяты лексемы с частотой выше 5.txt',
            'name': 'Изъяты лексемы с частотой выше 5'
        },
        {
            'path1': '../dataset/сокращение по частотности/1ё_Изъяты лексемы с частотой выше 3.txt',
            'path2': '../dataset/сокращение по частотности/2ё_Изъяты лексемы с частотой выше 3.txt',
            'name': 'Изъяты лексемы с частотой выше 3'
        },
        
        
    ]
    
    # Procesar todos los conjuntos de datos
    results = []
    #for dataset in datasets:
#     for dataset in funciones.tqdm(datasets, total=len(datasets)):
#         result = funciones.train_and_evaluate_dataset(
#             dataset['path1'],
#             dataset['path2'],
#             config,
#             dataset['name']
#         )
#         results.append(result)
    

    args = [(dataset, config) for dataset in datasets]
 
    results = []
    for arg in tqdm(args, desc="Entrenando secuencial"):
        result = train_wrapper(arg)
        results.append(result)
            
    
    # Mostrar resumen final
    print("\nResumen de todos los experimentos:")
    for result in results:
        print(f"{result['dataset_name']}: "
              f"{result['avg_accuracy']:.4f} ± {result['std_accuracy']:.4f}")

        
if __name__ == "__main__":
    main()

2025-03-31 14:02:33.560679: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Entrenando secuencial:   0%|          | 0/6 [00:00<?, ?it/s]


Procesando conjunto de datos: Изъяты лексемы с частотой выше 100

Repetición con semilla 0
train_Изъяты лексемы с частотой выше 100.csv
test_Изъяты лексемы с частотой выше 100.csv

Repetición con semilla 1

Repetición con semilla 2

Repetición con semilla 3

Repetición con semilla 4

Repetición con semilla 5


Entrenando secuencial:  17%|█▋        | 1/6 [01:20<06:44, 80.91s/it]


Procesando conjunto de datos: Изъяты лексемы с частотой выше 49

Repetición con semilla 0
train_Изъяты лексемы с частотой выше 49.csv
test_Изъяты лексемы с частотой выше 49.csv

Repetición con semilla 1

Repetición con semilla 2

Repetición con semilla 3

Repetición con semilla 4

Repetición con semilla 5


Entrenando secuencial:  33%|███▎      | 2/6 [02:40<05:21, 80.42s/it]

...


Procesando conjunto de datos: Изъяты лексемы с частотой выше 9

Repetición con semilla 0
train_Изъяты лексемы с частотой выше 9.csv
test_Изъяты лексемы с частотой выше 9.csv

Repetición con semilla 1

Repetición con semilla 2

Repetición con semilla 3

Repetición con semilla 4

Repetición con semilla 5


Entrenando secuencial:  67%|██████▋   | 4/6 [05:09<02:31, 75.83s/it]


Procesando conjunto de datos: Изъяты лексемы с частотой выше 5

Repetición con semilla 0
train_Изъяты лексемы с частотой выше 5.csv
test_Изъяты лексемы с частотой выше 5.csv

Repetición con semilla 1

Repetición con semilla 2

Repetición con semilla 3

Repetición con semilla 4

Repetición con semilla 5


Entrenando secuencial:  83%|████████▎ | 5/6 [05:40<00:59, 59.73s/it]


Procesando conjunto de datos: Изъяты лексемы с частотой выше 3

Repetición con semilla 0
train_Изъяты лексемы с частотой выше 3.csv
test_Изъяты лексемы с частотой выше 3.csv

Repetición con semilla 1

Repetición con semilla 2


In [ ]:
import torch
num_gpus = torch.cuda.device_count() 
print(num_gpus)

In [ ]:
#batch_size=128